In [4]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [5]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: chain
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:06<00:00, 1509.17it/s]


{'basic': {'expected_sample_burst_size': 3.001391659321798,
           'expected_sample_data_size': 16.517326158556386,
           'expected_time_to_recover': 33.1856,
           'packet_size': 5,
           'payload_bits': 63.789577742910744,
           'std_time_to_recover': 8.11961530123195},
 'derived': {'bit_efficency': 0.4076361417087459,
             'bit_efficency_ratio': 0.38444131034491313,
             'bit_efficency_sigma': 0.10040280186327534,
             'bit_redundancy': 102.13842225708925,
             'ideal_bit_efficency': 0.45429880006762086,
             'ideal_bit_redundancy': 76.62368712465192,
             'ideal_packet_efficiency': 2.2714940003381043,
             'ideal_packet_redundancy': 15.324737424930385,
             'ideal_time_to_recover': 28.082652973512534,
             'ideal_transmitted_bits': 140.41326486756267,
             'packet_efficiency': 1.9222065517245654,
             'packet_redundancy': 20.427684451417853,
             'permeability': 0

benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1287.68it/s]


{'basic': {'expected_sample_burst_size': 3.0000789328281634,
           'expected_sample_data_size': 16.49360644091878,
           'expected_time_to_recover': 40.1087,
           'packet_size': 5,
           'payload_bits': 78.5102495297363,
           'std_time_to_recover': 9.512343786365166},
 'derived': {'bit_efficency': 0.4116553036902152,
             'bit_efficency_ratio': 0.39148738069165195,
             'bit_efficency_sigma': 0.09122867741722053,
             'bit_redundancy': 122.0332504702637,
             'ideal_bit_efficency': 0.4626963699135191,
             'ideal_bit_redundancy': 91.16959806537314,
             'ideal_packet_efficiency': 2.3134818495675957,
             'ideal_packet_redundancy': 18.233919613074622,
             'ideal_time_to_recover': 33.93596951902188,
             'ideal_transmitted_bits': 169.67984759510944,
             'packet_efficiency': 1.9574369034582597,
             'packet_redundancy': 24.40665009405274,
             'permeability': 0.8460

...

{'basic': {'expected_sample_burst_size': 2.9921545604952766,
           'expected_sample_data_size': 16.572076639803598,
           'expected_time_to_recover': 27.236800000000002,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.484012865310687},
 'derived': {'bit_efficency': 0.133726097282325,
             'bit_efficency_ratio': 0.11748810432943663,
             'bit_efficency_sigma': 0.04742598848037567,
             'bit_redundancy': 120.18400000000003,
             'ideal_bit_efficency': 0.13870105034785637,
             'ideal_bit_redundancy': 99.35601179567621,
             'ideal_packet_efficiency': 0.6935052517392819,
             'ideal_packet_redundancy': 19.871202359135243,
             'ideal_time_to_recover': 23.071202359135242,
             'ideal_transmitted_bits': 115.35601179567621,
             'packet_efficiency': 0.5874405216471832,
             'packet_redundancy': 24.036800000000003,
             'permeability': 0.

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 2900.20it/s]


{'basic': {'expected_sample_burst_size': 2.992282249173098,
           'expected_sample_data_size': 16.545650233632593,
           'expected_time_to_recover': 27.629199999999994,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.803513658065137},
 'derived': {'bit_efficency': 0.1321969571821964,
             'bit_efficency_ratio': 0.1158194953165492,
             'bit_efficency_sigma': 0.04722213806185282,
             'bit_redundancy': 122.14599999999996,
             'ideal_bit_efficency': 0.13676546087548683,
             'ideal_bit_redundancy': 100.98860149030332,
             'ideal_packet_efficiency': 0.6838273043774341,
             'ideal_packet_redundancy': 20.197720298060666,
             'ideal_time_to_recover': 23.397720298060666,
             'ideal_transmitted_bits': 116.98860149030332,
             'packet_efficiency': 0.5790974765827459,
             'packet_redundancy': 24.429199999999994,
             'permeability': 0.

benchmark: 100%|██████████| 10000/10000 [00:04<00:00, 2390.48it/s]


{'basic': {'expected_sample_burst_size': 2.9864449857036957,
           'expected_sample_data_size': 16.55639097744361,
           'expected_time_to_recover': 27.3053,
           'packet_size': 5,
           'payload_bits': 16.0,
           'std_time_to_recover': 10.483353085249012},
 'derived': {'bit_efficency': 0.13334316490569156,
             'bit_efficency_ratio': 0.11719336539060182,
             'bit_efficency_sigma': 0.04710013374839236,
             'bit_redundancy': 120.5265,
             'ideal_bit_efficency': 0.1383327271576277,
             'ideal_bit_redundancy': 99.66315743756198,
             'ideal_packet_efficiency': 0.6916636357881386,
             'ideal_packet_redundancy': 19.932631487512396,
             'ideal_time_to_recover': 23.132631487512395,
             'ideal_transmitted_bits': 115.66315743756198,
             'packet_efficiency': 0.5859668269530092,
             'packet_redundancy': 24.1053,
             'permeability': 0.8471846669881816,
             '

### LT vs ANFEXT2L benchmark sweep
This section benchmarks LT and `anfext2l_undegenerate` across multiple payload sizes, caches the distributions, and plots the expected time-to-recover (TTR) and bit efficiency per payload.

In [6]:
from __future__ import annotations

import importlib
import json
from datetime import datetime
from pathlib import Path
import math
import numpy as np

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from tqdm.auto import tqdm

import impls.systematic as systematic
import impls.raptor as raptor
from benchmark_utils import benchmark, compute_distribution_stats

# Always reload to pick up local code edits when rerunning the cell
importlib.reload(systematic)
importlib.reload(raptor)

# -----------------------------------------------------------------------------
# Configuration (adjust N or payload grids if you want to re-run with new params)
# -----------------------------------------------------------------------------
N = 7


def _payload_sizes_for_anfext2l(
    n: int, min_bits: int, max_bits: int, max_payload: int | None = None
) -> list[int]:
    base = (1 << n) - 2
    sizes: list[int] = []
    m = 1
    while True:
        payload = base**m
        bits = math.log2(payload)
        if bits > max_bits or (max_payload is not None and payload > max_payload):
            break
        if bits >= min_bits:
            sizes.append(payload)
        m += 1
    return sizes


ANF_OVERRIDE_FN = getattr(systematic, "override_D", None)
ANF_MAX_PAYLOAD = ANF_OVERRIDE_FN(N) if ANF_OVERRIDE_FN is not None else None
LT_OVERRIDE_FN = getattr(raptor, "override_D", None)

PAYLOAD_BITS_BY_SCHEME = {
    "raptor": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
    "systematic": [1 << bits for bits in range(N, 2 ** (N - 1) + 1)],
}
BENCHMARK_CFG = dict(passes=200, iters_bound=400, processes=None, progress_update=100)
PLOT_WIDTH = 1050  # 1.5x wider plots
RESULTS_PATH = Path("research/lt_vs_anfext2l_results.jsonl")
RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

SCHEMES = {
    "raptor": {
        "producer": raptor.producer_constructor,
        "recoverer": raptor.recoverer_constructor,
        "override": LT_OVERRIDE_FN,
    },
    "systematic": {
        "producer": systematic.producer_constructor,
        "recoverer": systematic.recoverer_constructor,
        "override": ANF_OVERRIDE_FN,
    },
}


def _rgba_from_hex(hex_color: str, alpha: float) -> str:
    hex_color = hex_color.lstrip("#")
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f"rgba({r}, {g}, {b}, {alpha})"


SCHEME_COLORS = {
    "raptor": "#1f77b4",
    "systematic": "#d62728",
}

SCHEME_SIGMA_FILLS = {
    scheme: {
        "1sigma": _rgba_from_hex(color, 0.35),
        "2sigma": _rgba_from_hex(color, 0.18),
    }
    for scheme, color in SCHEME_COLORS.items()
}


def _load_records(path: Path) -> list[dict]:
    if not path.exists():
        return []
    records: list[dict] = []
    with path.open("r", encoding="utf-8") as fh:
        for line in fh:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def _append_record(path: Path, record: dict) -> None:
    with path.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(record) + "\n")


records = _load_records(RESULTS_PATH)
existing_keys = {(rec["scheme"], rec["D"], rec["N"]) for rec in records}
plan = [
    (scheme, payload_size)
    for scheme, payloads in PAYLOAD_BITS_BY_SCHEME.items()
    for payload_size in payloads
]
pending = [
    (scheme, payload_size)
    for scheme, payload_size in plan
    if (scheme, payload_size, N) not in existing_keys
]

if pending:
    total_jobs = len(pending)
    print(f"Running {total_jobs} new benchmarks (N={N}).")
    with tqdm(total=total_jobs, desc=f"Benchmarks (N={N})") as bar:
        for scheme, payload_size in pending:
            impl = SCHEMES[scheme]
            D = payload_size
            payload_bits = math.log2(D)
            override = impl.get("override")
            if override is not None:
                max_d = override(N)
                if D > max_d:
                    raise ValueError(
                        f"Payload size D={D} (bits={payload_bits:.2f}) exceeds {scheme} support for N={N}"
                    )
            bench_result = benchmark(
                impl["producer"],
                impl["recoverer"],
                N,
                D,
                **BENCHMARK_CFG,
            )
            record = {
                "scheme": scheme,
                "payload_bits": payload_bits,
                "N": N,
                "D": D,
                "timestamp": datetime.utcnow().isoformat(),
                "benchmark_cfg": BENCHMARK_CFG,
                "result": bench_result,
            }
            records.append(record)
            _append_record(RESULTS_PATH, record)
            bar.update(1)
else:
    print(f"All requested payload benchmarks already cached for N={N}.")

records_df = pd.DataFrame(records)
records_df = records_df[records_df["N"] == N].copy()
if records_df.empty:
    raise RuntimeError(
        "No cached benchmark rows for the current N; run the cell again to generate them."
    )

summary_rows = []
for rec in records_df.to_dict("records"):
    stats = compute_distribution_stats(rec["N"], rec["D"], rec["result"])
    metrics = stats["metrics"]
    summary_rows.append(
        {
            "scheme": rec["scheme"],
            "payload_bits": rec["payload_bits"],
            "expected_ttr": metrics["basic"]["expected_time_to_recover"],
            "ttr_sigma": metrics["basic"].get("std_time_to_recover", float("nan")),
            "bit_efficiency": metrics["derived"]["bit_efficency"],
            "bit_efficiency_sigma": metrics["derived"].get(
                "bit_efficency_sigma", float("nan")
            ),
            "bit_redundancy": metrics["derived"].get("bit_redundancy", float("nan")),
            "ideal_bit_efficency": metrics["derived"].get(
                "ideal_bit_efficency", float("nan")
            ),
            "ideal_bit_redundancy": metrics["derived"].get(
                "ideal_bit_redundancy", float("nan")
            ),
            "ideal_packet_efficiency": metrics["derived"].get(
                "ideal_packet_efficiency", float("nan")
            ),
            "ideal_packet_redundancy": metrics["derived"].get(
                "ideal_packet_redundancy", float("nan")
            ),
            "ideal_time_to_recover": metrics["derived"].get(
                "ideal_time_to_recover", float("nan")
            ),
            "ideal_transmitted_bits": metrics["derived"].get(
                "ideal_transmitted_bits", float("nan")
            ),
            "packet_efficiency": metrics["derived"].get(
                "packet_efficiency", float("nan")
            ),
            "packet_redundancy": metrics["derived"].get(
                "packet_redundancy", float("nan")
            ),
        }
    )

summary_df = pd.DataFrame(summary_rows).sort_values(["scheme", "payload_bits"])
print("Cached benchmark summary:")
display(summary_df)

fig_ttr = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    expected_arr = group["expected_ttr"].to_numpy(dtype=float)
    sigma_arr = group["ttr_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = expected_arr + mult * sigma_arr
            lower = np.clip(expected_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma"
            fig_ttr.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_ttr.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["expected_ttr"],
            mode="lines+markers",
            name=f"{scheme} E[TTR]",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>E[TTR]=%{y:.2f} packets<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_ttr.update_layout(
    title=f"Expected time-to-recover vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="expected TTR (packets)",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_ttr.show()

fig_eff = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    payload_bits_arr = group["payload_bits"].to_numpy(dtype=float)
    eff_arr = group["bit_efficiency"].to_numpy(dtype=float)
    sigma_arr = group["bit_efficiency_sigma"].to_numpy(dtype=float)
    fills = SCHEME_SIGMA_FILLS.get(scheme, {})
    if payload_bits_arr.size and not np.all(np.isnan(sigma_arr)):
        sigma_arr = np.nan_to_num(sigma_arr, nan=0.0)
        for mult, fill_key in ((2, "2sigma"), (1, "1sigma")):
            fill_color = fills.get(fill_key)
            if fill_color is None:
                continue
            upper = eff_arr + mult * sigma_arr
            lower = np.clip(eff_arr - mult * sigma_arr, a_min=0, a_max=None)
            band_label = f"{scheme} +/- {mult} sigma (bit eff)"
            fig_eff.add_trace(
                go.Scatter(
                    x=np.concatenate([payload_bits_arr, payload_bits_arr[::-1]]),
                    y=np.concatenate([upper, lower[::-1]]),
                    fill="toself",
                    fillcolor=fill_color,
                    line=dict(color="rgba(0,0,0,0)"),
                    hoverinfo="skip",
                    showlegend=bool(mult == 1),
                    legendgroup=f"{scheme}_eff_bands",
                    name=band_label,
                )
            )
    color = SCHEME_COLORS.get(scheme)
    fig_eff.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["bit_efficiency"],
            mode="lines+markers",
            name=f"{scheme} bit efficiency",
            line=dict(shape="spline", color=color),
            marker=dict(color=color),
            legendgroup=scheme,
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>efficiency=%{y:.4f}<extra></extra>",
            text=[scheme] * len(group),
        )
    )
fig_eff.update_layout(
    title=f"Bit efficiency vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="payload bits per transmitted bit",
    width=PLOT_WIDTH,
    template="plotly_white",
)
fig_eff.show()

ADDITIONAL_METRICS = [
    ("bit_redundancy", "Bit redundancy vs payload size", "extra bits"),
    (
        "ideal_bit_efficency",
        "Ideal bit efficiency vs payload size",
        "ideal bits per transmitted bit",
    ),
    (
        "ideal_bit_redundancy",
        "Ideal bit redundancy vs payload size",
        "ideal extra bits",
    ),
    (
        "ideal_packet_efficiency",
        "Ideal packet efficiency vs payload size",
        "payload bits per ideal packet",
    ),
    (
        "ideal_packet_redundancy",
        "Ideal packet redundancy vs payload size",
        "ideal extra packets",
    ),
    ("ideal_time_to_recover", "Ideal time-to-recover vs payload size", "ideal packets"),
    (
        "ideal_transmitted_bits",
        "Ideal transmitted bits vs payload size",
        "ideal transmitted bits",
    ),
    (
        "packet_efficiency",
        "Packet efficiency vs payload size",
        "payload bits per packet",
    ),
    ("packet_redundancy", "Packet redundancy vs payload size", "extra packets"),
]

for metric_key, title, y_axis in ADDITIONAL_METRICS:
    if metric_key not in summary_df.columns:
        continue
    fig = go.Figure()
    for scheme, group in summary_df.groupby("scheme"):
        group = group.sort_values("payload_bits")
        color = SCHEME_COLORS.get(scheme)
        fig.add_trace(
            go.Scatter(
                x=group["payload_bits"],
                y=group[metric_key],
                mode="lines+markers",
                name=f"{scheme} {metric_key}",
                line=dict(shape="spline", color=color),
                marker=dict(color=color),
                legendgroup=scheme,
                hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>value=%{y:.4f}<extra></extra>",
                text=[scheme] * len(group),
            )
        )
    fig.update_layout(
        title=f"{title} (N={N})",
        xaxis_title="payload bits",
        yaxis_title=y_axis,
        template="plotly_white",
        width=PLOT_WIDTH,
    )
    fig.show()

Running 116 new benchmarks (N=7).


Benchmarks (N=7): 100%|██████████| 116/116 [03:35<00:00,  1.86s/it]


Cached benchmark summary:


,scheme,payload_bits,expected_ttr,ttr_sigma,bit_efficiency,bit_efficiency_sigma,bit_redundancy,ideal_bit_efficency,ideal_bit_redundancy,ideal_packet_efficiency,ideal_packet_redundancy,ideal_time_to_recover,ideal_transmitted_bits,packet_efficiency,packet_redundancy
0,raptor,7.0,3.955,1.372215,0.291000,0.115972,20.685,0.298500,16.450571,2.089501,2.350082,3.350082,23.450571,1.769912,2.955000
1,raptor,8.0,5.840,1.853213,0.217683,0.074847,32.880,0.229207,26.902924,1.604450,3.843275,4.986132,34.902924,1.369863,4.697143
2,raptor,9.0,5.850,1.818653,0.242125,0.078320,31.950,0.259566,25.673298,1.816960,3.667614,4.953328,34.673298,1.538462,4.564286
3,raptor,10.0,6.165,1.933333,0.258060,0.090867,33.155,0.273898,26.509951,1.917286,3.787136,5.215707,36.509951,1.622060,4.736429
4,raptor,11.0,6.000,2.044505,0.292955,0.101080,31.000,0.311145,24.353312,2.178014,3.479045,5.050473,35.353312,1.833333,4.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,systematic,60.0,108.990,26.253569,0.083046,0.019147,702.930,0.092790,586.618583,0.649533,83.802655,92.374083,646.618583,0.550509,100.418571
112,systematic,61.0,116.575,28.173469,0.078945,0.018127,755.025,0.088203,630.589451,0.617418,90.084207,98.798493,691.589451,0.523268,107.860714
113,systematic,62.0,114.395,27.808973,0.081767,0.018780,738.765,0.091220,617.678126,0.638538,88.239732,97.096875,679.678126,0.541982,105.537857
114,systematic,63.0,163.400,48.004166,0.059398,0.015862,1080.800,0.065249,902.536026,0.456741,128.933718,137.933718,965.536026,0.385557,154.400000
